# Reflexion Prompting Example

This notebook demonstrates **Reflexion**, a powerful technique for self-correction and iterative improvement.

**The Concept:**
Humans rarely get complex problems right on the first try. We draft, review, catch errors, and revise. **Reflexion** gives the AI this same ability. Instead of accepting the first answer, the model:
1.  **Attempts** to solve the problem.
2.  **Evaluates** its own work (or receives feedback).
3.  **Reflects** on *why* it might be wrong (e.g., "I missed a constraint").
4.  **Retries** with this new insight.

**Key Mechanics:**
* **The Solver**: A model instance trying to answer the query.
* **The Reflector**: A model instance (often the same one) critiquing the previous attempt.
* **The Loop**: A cycle of *Attempt → Critique → Reflect → Retry* that continues until a valid answer is found or a limit is reached.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

client = OpenAI(api_key=api_key)
print("OpenAI client ready! Using gpt-4o-mini (excellent at self-correction)")

### 2. The Reflexion Engine

This function encapsulates the entire self-correction loop.

**How it works:**
1.  **History**: We keep a list of `reflections` (lessons learned from past failures).
2.  **Prompting**: In every attempt, we inject the *previous reflections* into the prompt. This forces the model to specifically avoid repeating its past mistakes.
3.  **Self-Check**: If the model doesn't output a specific success tag (`<FINAL_ANSWER>`), we assume it's still reasoning or unsure, and we trigger a specific "Reflection Prompt" to analyze what went wrong.

In [ ]:
def reflexion_solve(problem: str, max_reflections: int = 3):
    reflections = []
    attempts = []

    print(f"Problem: {problem}\n")
    print("Starting Reflexion Agent...\n")
    time.sleep(1)

    for attempt in range(1, max_reflections + 2):
        print(f"--- ATTEMPT {attempt} ---")
        
        # 1. Construct the Prompt with History
        # If we have past reflections, we add them to the context.
        reflection_context = ""
        if reflections:
            reflection_context = (
                "Previous reflections on mistakes:\n" + 
                "\n".join(reflections[-3:]) + 
                "\n\nUse these insights to avoid repeating errors.\n"
            )

        prompt = f"""
{reflection_context}

Solve this problem step by step:
{problem}

Think carefully.
If you are confident you have the correct solution, end your response with:
<FINAL_ANSWER> [your answer here] </FINAL_ANSWER>
"""

        try:
            # 2. Generate the Attempt
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a reasoning agent. Learn from past mistakes to improve."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7 if attempt == 1 else 0.5  # Lower temp on retries for focus
            )
            
            reply = response.choices[0].message.content
            print(f"Model Output:\n{reply[:200]}...\n(truncated for brevity)\n")
            attempts.append(reply)

            # 3. Check for Success
            if "<FINAL_ANSWER>" in reply:
                answer = reply.split("<FINAL_ANSWER>")[1].split("</FINAL_ANSWER>")[0].strip()
                print(f"\n✅ SUCCESS! Final Answer: {answer}\n")
                return answer, attempts, reflections

            # 4. Trigger Reflection (if failed)
            if attempt <= max_reflections:
                print("\n⚠️ No valid final answer detected. Triggering self-reflection...")
                
                reflect_prompt = f"""
The previous attempt failed to produce a confident final answer.

Here was the attempt:
{reply}

Reflect deeply:
- What went wrong?
- What assumptions were incorrect?
- What step was missed?
- How should you reason differently next time?

Be concise and honest.
"""
                reflect_response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "user", "content": reflect_prompt}],
                    temperature=0.7
                )

                reflection = reflect_response.choices[0].message.content.strip()
                reflections.append(f"Reflection {len(reflections)+1}: {reflection}")
                print(f"🤔 Reflection: {reflection}\n")
                time.sleep(1)
                
        except Exception as e:
            print(f"Error during attempt {attempt}: {e}")
            continue

    print("❌ Max reflections reached without a final answer.")
    return None, attempts, reflections

### 3. Testing with Hard Problems

Reflexion shines on problems that are "tricky"—where an impulsive answer is often wrong.

**Examples provided:**
1.  **Logic Puzzle**: The "Three Switch" problem (requires physical reasoning).
2.  **Math Riddle**: The "Lily Pad" problem (requires exponential reasoning).
3.  **Constraint Satisfaction**: Einstein's Riddle (requires keeping track of many variables).

In [ ]:
# Pre-defined tricky problems
problems = {
    "1": """
You have 3 light bulbs in a room and 3 switches outside. Each switch controls one bulb.
One switch turns it on/off, one does nothing, one turns it on only when the door is closed.
You can enter the room only once.
How do you determine which switch does what?
""",
    "2": """
A lily pad doubles in size every day. On day 30, it covers the entire pond.
On what day did it cover half the pond?
""",
    "3": """
Solve this riddle:
I speak without a mouth and hear without ears. I have no body, but I come alive with wind. What am I?
"""
}

print("Choose a problem to test Reflexion:")
print("1. Light Bulb Puzzle")
print("2. Lily Pad Math")
print("3. What Am I?")
print("4. Custom Input")

choice = input("\nEnter 1-4: ").strip()

if choice == "4":
    selected_problem = input("Enter your problem: ")
else:
    selected_problem = problems.get(choice, problems["1"])

# Run the Agent
final_ans, _, _ = reflexion_solve(selected_problem, max_reflections=3)